In [1]:
!pip install pandas
!pip install numpy
!pip install torch
!pip install tqdm
!pip install matplotlib
!pip install scipy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is n

In [2]:
from os import listdir
import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader

import Sources.Utils as UtilsTool
import Sources.Attacks as AttacksTool
import Sources.Data as DataTool
from Sources.Models import RNNModel, GRUModel, LSTMModel, LinearDegradationModel, LogDegradationModel, ExpDegradationModel

In [3]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

DisplayGraph = 0
TrainMode = 0
OptimizeParameters = 0

DatasetPath = 'DatasetRUL/'
ModelPath = 'Models/'
NormalOutputPath = 'ResultsRUL/Normal/'
AdversarialOutputPath = 'ResultsRUL/Adversarial/'
OptimalParametersPath = 'ResultsRUL/OptimalParameters/'
FiguresPath = 'ResultsRUL/Figures/'

window = 80

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
listdir(DatasetPath)

df_train = pd.read_csv(DatasetPath + 'train_FD001.txt', header = None, sep = ' ')
df_test = pd.read_csv(DatasetPath + 'test_FD001.txt', header = None, sep = ' ')
rul_test = pd.read_csv(DatasetPath + 'RUL_FD001.txt', header = None)

In [6]:
df_train, df_test, rul_test = DataTool.SetDatasets(df_train, df_test, rul_test)

In [7]:
minmax_dict = DataTool.GetMinMaxDictionnary(df_train)
Ndf_train  = DataTool.NormalizeDataset(df_train, minmax_dict)    
Ndf_test  = DataTool.NormalizeDataset(df_test, minmax_dict)   

In [8]:
n_features = len([c for c in Ndf_train.columns if 's' in c])

In [9]:
np.random.seed(5)
torch.manual_seed(5)

units = np.arange(1,101)
train_units = list(np.random.choice(units, 80, replace = False))
val_units = list(set(units) - set(train_units))

train_data = Ndf_train[Ndf_train['unit'].isin(train_units)].copy()
val_data = Ndf_train[Ndf_train['unit'].isin(val_units)].copy()

train_indices = list(train_data[(train_data['rul'] >= (window - 1)) & (train_data['time'] > 10)].index)
val_indices = list(val_data[(val_data['rul'] >= (window - 1)) & (val_data['time'] > 10)].index)

train = DataTool.data(train_indices, Ndf_train, window)
val = DataTool.data(val_indices, Ndf_train, window)

UnitBySize = Ndf_test.groupby('unit')['time'].max().tolist()
Units = []
for i in range(len(UnitBySize)):
    if UnitBySize[i] > window:
        Units.append(i+1)

testdataset = DataTool.test(Ndf_test, window)

In [10]:
print("Train", len(train))
print("Val", len(val))
print("Test", len(testdataset))

Train 9615
Val 2116
Test 77


In [11]:
trainloader = DataLoader(train, batch_size = 64, shuffle = True)
valloader = DataLoader(val, batch_size = len(val_indices), shuffle = True)
testloader = DataLoader(testdataset, batch_size = 100)

In [12]:
if TrainMode == 1:
    epochs = 30
    learning_rate= 0.001
    
    MyRNNModel = RNNModel(n_features, 12).to(device)
    UtilsTool.InitModel(MyRNNModel, device)
    UtilsTool.TrainModel(MyRNNModel, trainloader, valloader, epochs, learning_rate, device)        
    torch.save(MyRNNModel, ModelPath + 'MyRNNmodel.pth')

    MyGRUModel = GRUModel(n_features, 12).to(device)
    UtilsTool.InitModel(MyGRUModel, device)
    UtilsTool.TrainModel(MyGRUModel, trainloader, valloader, epochs, learning_rate, device)        
    torch.save(MyGRUModel, ModelPath + 'MyGRUmodel.pth')

    MyLSTMModel = LSTMModel(n_features, 12).to(device)
    UtilsTool.InitModel(MyLSTMModel, device)
    UtilsTool.TrainModel(MyLSTMModel, trainloader, valloader, epochs, learning_rate, device)     
    torch.save(MyLSTMModel, ModelPath + 'MyLSTMmodel.pth')
   
    epochs = 30
    learning_rate= 0.1

    MyLinearModel = LinearDegradationModel(24,80).cuda().float()
    UtilsTool.InitModel(MyLinearModel, device)
    UtilsTool.TrainModel(MyLinearModel, trainloader, valloader, epochs, learning_rate, device)   
    torch.save(MyLinearModel, ModelPath + 'MyLinearmodel.pth')     

    MyLogModel = LogDegradationModel(24,80).cuda().float() 
    UtilsTool.InitModel(MyLogModel, device)  
    UtilsTool.TrainModel(MyLogModel, trainloader, valloader, epochs, learning_rate, device)    
    torch.save(MyLogModel, ModelPath + 'MyLogmodel.pth')   
 
    epochs = 40
    learning_rate= 0.1
    
    MyExpModel = ExpDegradationModel(24,80).cuda().float()
    UtilsTool.InitModel(MyExpModel, device)   
    UtilsTool.TrainModel(MyExpModel, trainloader, valloader, epochs, learning_rate, device)       
    torch.save(MyExpModel, ModelPath + 'MyExpmodel.pth')  
    
else:     
    MyRNNModel = torch.load(ModelPath + 'MyRNNmodel.pth')
    MyGRUModel = torch.load(ModelPath + 'MyGRUmodel.pth')
    MyLSTMModel = torch.load(ModelPath + 'MyLSTMmodel.pth')
    MyLinearModel = torch.load(ModelPath + 'MyLinearmodel.pth')
    MyLogModel = torch.load(ModelPath + 'MyLogmodel.pth')
    MyExpModel = torch.load(ModelPath + 'MyExpmodel.pth')


In [13]:
MyRNNModel = MyRNNModel.to(device)
MyGRUModel = MyGRUModel.to(device)
MyLSTMModel = MyLSTMModel.to(device)
MyLinearModel = MyLinearModel.to(device).float()
MyLogModel = MyLogModel.to(device).float()
MyExpModel = MyExpModel.to(device).float()

![](./animation3.gif)

In [14]:
mse, l1, Rrmse, y_pred_rnn, y = UtilsTool.test(MyRNNModel, testloader, device)
print(f'RNN: MSE:{round(mse,2)}, L1:{round(l1,2)}, RMSE:{round(Rrmse,2)}')
mse, l1, Rrmse, y_pred_gru, y = UtilsTool.test(MyGRUModel, testloader, device)
print(f'GRU: MSE:{round(mse,2)}, L1:{round(l1,2)}, RMSE:{round(Rrmse,2)}')
mse, l1, Rrmse, y_pred_lstm, y = UtilsTool.test(MyLSTMModel, testloader, device)
print(f'LSTM: MSE:{round(mse,2)}, L1:{round(l1,2)}, RMSE:{round(Rrmse,2)}')
mse, l1, Rrmse, y_pred_linear, y = UtilsTool.test(MyLinearModel, testloader, device)
print(f'Linear Model: MSE:{round(mse,2)}, L1:{round(l1,2)}, RMSE:{round(Rrmse,2)}')
mse, l1, Rrmse, y_pred_log, y = UtilsTool.test(MyLogModel, testloader, device)
print(f'Log Model: MSE:{round(mse,2)}, L1:{round(l1,2)}, RMSE:{round(Rrmse,2)}')
mse, l1, Rrmse, y_pred_exp, y = UtilsTool.test(MyExpModel, testloader, device)
print(f'Exp Model: MSE:{round(mse,2)}, L1:{round(l1,2)}, RMSE:{round(Rrmse,2)}')

RNN: MSE:459.84, L1:14.86, RMSE:21.44
GRU: MSE:187.28, L1:9.44, RMSE:13.69
LSTM: MSE:239.46, L1:10.61, RMSE:15.47
Linear Model: MSE:377.44, L1:16.24, RMSE:19.43
Log Model: MSE:550.46, L1:17.73, RMSE:23.46
Exp Model: MSE:453.13, L1:16.27, RMSE:21.29


In [15]:
if DisplayGraph == 1:
    UtilsTool.PlotTensor(MyLinearModel.a.detach().cpu(), "LinearA")
    UtilsTool.PlotTensor(MyLogModel.a.detach().cpu(), "LinearA")
    UtilsTool.PlotTensor(MyExpModel.a.detach().cpu(), "LinearA")

In [16]:
if DisplayGraph == 1:
    UtilsTool.DisplayGraph(y_pred_rnn, y, FiguresPath, "NormalRNNDetection")
    UtilsTool.DisplayGraph(y_pred_gru, y, FiguresPath, "NormalGRUDetection")
    UtilsTool.DisplayGraph(y_pred_lstm, y, FiguresPath, "NormalLSTMDetection")
    UtilsTool.DisplayGraph(y_pred_linear, y, FiguresPath, "NormalLinearDetection")
    UtilsTool.DisplayGraph(y_pred_log, y, FiguresPath, "NormalLogDetection")
    UtilsTool.DisplayGraph(y_pred_exp, y, FiguresPath, "NormalExpDetection")

In [17]:
testloaderAttack = DataLoader(testloader.dataset, batch_size=100, shuffle=False)
X, y = next(iter(testloaderAttack))
X, y = X.to(device), y.to(device)

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.06
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 120,
        "Epsilon": 0.001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.02,
    }
}

print("RNN")
RNNAdversarialDataFgsm, RNNAdversarialDataBim, RNNAdversarialDataCW = AttacksTool.TestAttacks(MyRNNModel, X, y, AttacksParameters, device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.060
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 65,
        "Epsilon": 0.001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.002,
    }
}

print("GRU")
GRUAdversarialDataFgsm, GRUAdversarialDataBim, GRUAdversarialDataCW = AttacksTool.TestAttacks(MyGRUModel, X, y, AttacksParameters, device)
print()

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.055
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 90,
        "Epsilon": 0.001
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.004,
    }
}

print("LSTM")
LSTMAdversarialDataFgsm, LSTMAdversarialDataBim, LSTMAdversarialDataCW = AttacksTool.TestAttacks(MyLSTMModel, X, y, AttacksParameters, device)

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.055
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 80,
        "Epsilon": 0.01
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.005,
    }
}
print()
print("LINEAR")
LinearAdversarialDataFgsm, LinearAdversarialDataBim, LinearAdversarialDataCW = AttacksTool.TestAttacks(MyLinearModel, X, y, AttacksParameters, device)

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.055
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 100,
        "Epsilon": 0.01
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.001,
    }
}
print()
print("LOG")
LogAdversarialDataFgsm, LogAdversarialDataBim, LogAdversarialDataCW = AttacksTool.TestAttacks(MyLogModel, X, y, AttacksParameters, device)

AttacksParameters = {
    "FGSM": {
        "Objective": 300,
        "Epsilon": 0.06
    },
    "BIM": {
        "Objective": 300,
        "Iterations": 80,
        "Epsilon": 0.05
    },
    "CW": {
        "Objective": 0,
        "Iterations": 1000,
        "LearningRate": 0.01,
        "c": 0.004,
    }
}
print()
print("EXP")
ExpAdversarialDataFgsm, ExpAdversarialDataBim, ExpAdversarialDataCW = AttacksTool.TestAttacks(MyExpModel, X, y, AttacksParameters, device)

RNN
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 41.668636322021484, 'RMSE_adversarial': 0.05490428954362869, 'RMSE_pred': 31.824527740478516, 'L0': tensor(24.)}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 21.016260147094727, 'RMSE_adversarial': 0.051802560687065125, 'RMSE_pred': 51.003746032714844, 'L0': tensor(24.)}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 0.0031628403812646866, 'RMSE_adversarial': 0.04933729022741318, 'RMSE_pred': 75.99961853027344, 'L0': tensor(24.)}

GRU
FGSM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 18.43494415283203, 'RMSE_adversarial': 0.054831214249134064, 'RMSE_pred': 57.1057014465332, 'L0': tensor(24.)}
BIM: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 2.5615804195404053, 'RMSE_adversarial': 0.05220765992999077, 'RMSE_pred': 72.80780792236328, 'L0': tensor(23.9351)}
CW: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 0.3970373868942261, 'RMSE_adversarial': 0.05328387

In [18]:
print("RNN")
PRNNAdversarialDataFgsm = UtilsTool.TestProjectionCost(testloaderAttack, X, RNNAdversarialDataFgsm, MyRNNModel, minmax_dict, window, device)
PRNNAdversarialDataBim = UtilsTool.TestProjectionCost(testloaderAttack, X, RNNAdversarialDataBim, MyRNNModel, minmax_dict, window, device)
PRNNAdversarialDataCW = UtilsTool.TestProjectionCost(testloaderAttack, X, RNNAdversarialDataCW, MyRNNModel, minmax_dict, window, device)

print("GRU")
PGRUAdversarialDataFgsm = UtilsTool.TestProjectionCost(testloaderAttack, X, GRUAdversarialDataFgsm, MyGRUModel, minmax_dict, window, device)
PGRUAdversarialDataBim = UtilsTool.TestProjectionCost(testloaderAttack, X, GRUAdversarialDataBim, MyGRUModel, minmax_dict, window, device)
PGRUAdversarialDataCW = UtilsTool.TestProjectionCost(testloaderAttack, X, GRUAdversarialDataCW, MyGRUModel, minmax_dict, window, device)

print("LSTM")
PLSTMAdversarialDataFgsm = UtilsTool.TestProjectionCost(testloaderAttack, X, LSTMAdversarialDataFgsm, MyLSTMModel, minmax_dict, window, device)
PLSTMAdversarialDataBim = UtilsTool.TestProjectionCost(testloaderAttack, X, LSTMAdversarialDataBim, MyLSTMModel, minmax_dict, window, device)
PLSTMAdversarialDataCW = UtilsTool.TestProjectionCost(testloaderAttack, X, LSTMAdversarialDataCW, MyLSTMModel, minmax_dict, window, device)

print("LINEAR")
PLinearAdversarialDataFgsm = UtilsTool.TestProjectionCost(testloaderAttack, X, LinearAdversarialDataFgsm, MyLinearModel, minmax_dict, window, device)
PLinearAdversarialDataBim = UtilsTool.TestProjectionCost(testloaderAttack, X, LinearAdversarialDataBim, MyLinearModel, minmax_dict, window, device)
PLinearAdversarialDataCW = UtilsTool.TestProjectionCost(testloaderAttack, X, LinearAdversarialDataCW, MyLinearModel, minmax_dict, window, device)

print("LOG")
PLogAdversarialDataFgsm = UtilsTool.TestProjectionCost(testloaderAttack, X, LogAdversarialDataFgsm, MyLogModel, minmax_dict, window, device)
PLogAdversarialDataBim = UtilsTool.TestProjectionCost(testloaderAttack, X, LogAdversarialDataBim, MyLogModel, minmax_dict, window, device)
PLogAdversarialDataCW = UtilsTool.TestProjectionCost(testloaderAttack, X, LogAdversarialDataCW, MyLogModel, minmax_dict, window, device)

print("EXP")
PExpAdversarialDataFgsm = UtilsTool.TestProjectionCost(testloaderAttack, X, ExpAdversarialDataFgsm, MyExpModel, minmax_dict, window, device)
PExpAdversarialDataBim = UtilsTool.TestProjectionCost(testloaderAttack, X, ExpAdversarialDataBim, MyExpModel, minmax_dict, window, device)
PExpAdversarialDataCW = UtilsTool.TestProjectionCost(testloaderAttack, X, ExpAdversarialDataCW, MyExpModel, minmax_dict, window, device)

RNN
Initial: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 41.668636322021484, 'RMSE_adversarial': 0.05490428954362869, 'RMSE_pred': 31.824527740478516, 'L0': tensor(24.)}
Formated: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 43.07719802856445, 'RMSE_adversarial': 0.05165189132094383, 'RMSE_pred': 30.74812889099121, 'L0': tensor(16.)}

Initial: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 21.016260147094727, 'RMSE_adversarial': 0.051802560687065125, 'RMSE_pred': 51.003746032714844, 'L0': tensor(24.)}
Formated: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 23.815214157104492, 'RMSE_adversarial': 0.04548564553260803, 'RMSE_pred': 48.335350036621094, 'L0': tensor(16.)}

Initial: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 0.0031628403812646866, 'RMSE_adversarial': 0.04933729022741318, 'RMSE_pred': 75.99961853027344, 'L0': tensor(24.)}
Formated: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 1.9371377229690552, 'RMSE_adv

In [19]:
UtilsTool.CreateDFSFiles(testloaderAttack, X, PRNNAdversarialDataFgsm, y, MyRNNModel, minmax_dict, NormalOutputPath + "/RNN", AdversarialOutputPath + "/RNN" + "/FGSM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PRNNAdversarialDataBim, y, MyRNNModel, minmax_dict, NormalOutputPath + "/RNN", AdversarialOutputPath + "/RNN" + "/BIM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PRNNAdversarialDataCW, y, MyRNNModel, minmax_dict, NormalOutputPath + "/RNN", AdversarialOutputPath + "/RNN" + "/CW", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, PGRUAdversarialDataFgsm, y, MyGRUModel, minmax_dict, NormalOutputPath + "/GRU", AdversarialOutputPath + "/GRU" + "/FGSM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PGRUAdversarialDataBim, y, MyGRUModel, minmax_dict, NormalOutputPath + "/GRU", AdversarialOutputPath + "/GRU" + "/BIM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PGRUAdversarialDataCW, y, MyGRUModel, minmax_dict, NormalOutputPath + "/GRU", AdversarialOutputPath + "/GRU" + "/CW", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, PLSTMAdversarialDataFgsm, y, MyLSTMModel, minmax_dict, NormalOutputPath + "/LSTM", AdversarialOutputPath + "/LSTM" + "/FGSM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PLSTMAdversarialDataBim, y, MyLSTMModel, minmax_dict, NormalOutputPath + "/LSTM", AdversarialOutputPath + "/LSTM" + "/BIM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PLSTMAdversarialDataCW, y, MyLSTMModel, minmax_dict, NormalOutputPath + "/LSTM", AdversarialOutputPath + "/LSTM" + "/CW", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, PLinearAdversarialDataFgsm, y, MyLinearModel, minmax_dict, NormalOutputPath + "/LINEAR", AdversarialOutputPath + "/LINEAR" + "/FGSM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PLinearAdversarialDataBim, y, MyLinearModel, minmax_dict, NormalOutputPath + "/LINEAR", AdversarialOutputPath + "/LINEAR" + "/BIM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PLinearAdversarialDataCW, y, MyLinearModel, minmax_dict, NormalOutputPath + "/LINEAR", AdversarialOutputPath + "/LINEAR" + "/CW", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, PLogAdversarialDataFgsm, y, MyLogModel, minmax_dict, NormalOutputPath + "/LOG", AdversarialOutputPath + "/LOG" + "/FGSM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PLogAdversarialDataBim, y, MyLogModel, minmax_dict, NormalOutputPath + "/LOG", AdversarialOutputPath + "/LOG" + "/BIM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PLogAdversarialDataCW, y, MyLogModel, minmax_dict, NormalOutputPath + "/LOG", AdversarialOutputPath + "/LOG" + "/CW", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, PExpAdversarialDataFgsm, y, MyExpModel, minmax_dict, NormalOutputPath + "/EXP", AdversarialOutputPath + "/EXP" + "/FGSM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PExpAdversarialDataBim, y, MyExpModel, minmax_dict, NormalOutputPath + "/EXP", AdversarialOutputPath + "/EXP" + "/BIM", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PExpAdversarialDataCW, y, MyExpModel, minmax_dict, NormalOutputPath + "/EXP", AdversarialOutputPath + "/EXP" + "/CW", device)

In [20]:
if DisplayGraph == 1:
    y_pred = MyRNNModel(PRNNAdversarialDataFgsm)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "FGSMRNNDetection")
    y_pred = MyRNNModel(PRNNAdversarialDataBim)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "BIMRNNDetection")
    y_pred = MyRNNModel(PRNNAdversarialDataCW)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "CWRNNDetection")

In [21]:
if DisplayGraph == 1:
    y_pred = MyGRUModel(PGRUAdversarialDataFgsm)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "FGSMGRUDetection")
    y_pred = MyGRUModel(PGRUAdversarialDataBim)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "BIMGRUDetection")
    y_pred = MyGRUModel(PGRUAdversarialDataCW)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "CWGRUDetection")

In [22]:
if DisplayGraph == 1:
    y_pred = MyLSTMModel(PLSTMAdversarialDataFgsm)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "FGSMLSTMDetection")
    y_pred = MyLSTMModel(PLSTMAdversarialDataBim)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "BIMLSTMDetection")
    y_pred = MyLSTMModel(PLSTMAdversarialDataCW)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "CWLSTMDetection")

In [23]:
if DisplayGraph == 1:
    y_pred = MyLinearModel(PLinearAdversarialDataFgsm)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "FGSMLinearDetection")
    y_pred = MyLinearModel(PLinearAdversarialDataBim)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "BIMLinearDetection")
    y_pred = MyLinearModel(PLinearAdversarialDataCW)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "CWLinearDetection")

In [24]:
if DisplayGraph == 1:
    y_pred = MyLogModel(PLogAdversarialDataFgsm)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "FGSMLogDetection")
    y_pred = MyLogModel(PLogAdversarialDataBim)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "BIMLogDetection")
    y_pred = MyLogModel(PLogAdversarialDataCW)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "CWLogDetection")

In [25]:
if DisplayGraph == 1:
    y_pred = MyExpModel(PExpAdversarialDataFgsm)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "FGSMExpDetection")
    y_pred = MyExpModel(PExpAdversarialDataBim)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "BIMExpDetection")
    y_pred = MyExpModel(PExpAdversarialDataCW)
    UtilsTool.DisplayGraph(y_pred, y, FiguresPath, "CWExpDetection")

In [26]:
if OptimizeParameters == 1:
    AttacksTool.OptimizeParameters(X, y, MyRNNModel, OptimalParametersPath, 2, "RNN", device)
    AttacksTool.OptimizeParameters(X, y, MyGRUModel, OptimalParametersPath, 2, "GRU", device)
    AttacksTool.OptimizeParameters(X, y, MyLSTMModel, OptimalParametersPath, 2, "LSTM", device)
    AttacksTool.OptimizeParameters(X, y, MyLogModel, OptimalParametersPath, 2, "Log", device)
    AttacksTool.OptimizeParameters(X, y, MyLinearModel, OptimalParametersPath, 2, "Linear", device)
    AttacksTool.OptimizeParameters(X, y, MyExpModel, OptimalParametersPath, 2, "Exp", device)

In [27]:
if DisplayGraph == 1:
    UtilsTool.AnalyzeK(5, OptimalParametersPath + "OptimizeKLinear.txt", OptimalParametersPath)   
    UtilsTool.AnalyzePS(5, OptimalParametersPath + "OptimizePSRNN.txt", OptimalParametersPath)   

In [28]:

if DisplayGraph == 1 and OptimizeParameters == 1:
    AttacksParametersMaillard = {
        "Maillard": {
            "Objective": 0,
            "Iterations": 1000,
            "LearningRate": 0.01,
            "Initial_c": 1,
            "Final_c": 0.001,
            "Steps_c":2,
            "k": 2,
            "P": 8,
            "S": 0.1
        }
    }
    AttacksTool.TestNewAttacks(MyRNNModel, X, y, AttacksParametersMaillard, device, FiguresPath, "NewAttackRNN_C_")

    AttacksParametersMaillard = {
        "Maillard": {
            "Objective": 0,
            "Iterations": 1000,
            "LearningRate": 0.01,
            "Initial_c": 1,
            "Final_c": 0.001,
            "Steps_c":2,
            "k": 2,
            "P": 8,
            "S": 0.1

        }
    }
    AttacksTool.TestNewAttacks(MyGRUModel, X, y, AttacksParametersMaillard, device, FiguresPath, "NewAttackGRU_C_")

    AttacksParametersMaillard = {
        "Maillard": {
            "Objective": 0,
            "Iterations": 1000,
            "LearningRate": 0.01,
            "Initial_c": 1,
            "Final_c": 0.001,
            "Steps_c":2,
            "k": 2,
            "P": 8,
            "S": 0.3

        }
    }
    AttacksTool.TestNewAttacks(MyLSTMModel, X, y, AttacksParametersMaillard, device, FiguresPath, "NewAttackLSTM_C_")

    AttacksParametersMaillard = {
        "Maillard": {
            "Objective": 0,
            "Iterations": 1000,
            "LearningRate": 0.01,
            "Initial_c": 1,
            "Final_c": 0.0001,
            "Steps_c":2,
            "k": 2,
            "P": 5,
            "S": 0.1

        }
    }
    AttacksTool.TestNewAttacks(MyLinearModel, X, y, AttacksParametersMaillard, device, FiguresPath, "NewAttackLinear_C_")

    AttacksParametersMaillard = {
        "Maillard": {
            "Objective": 0,
            "Iterations": 1000,
            "LearningRate": 0.01,
            "Initial_c": 1,
            "Final_c": 0.0001,
            "Steps_c":2,
            "k": 2,
            "P": 5,
            "S": 0.1

        }
    }
    AttacksTool.TestNewAttacks(MyLogModel, X, y, AttacksParametersMaillard, device, FiguresPath, "NewAttackLog_C_")

    AttacksParametersMaillard = {
        "Maillard": {
            "Objective": 0,
            "Iterations": 1000,
            "LearningRate": 0.01,
            "Initial_c": 1,
            "Final_c": 0.0001,
            "Steps_c":2,
            "k": 2,
            "P": 5,
            "S": 0.1

        }
    }
    AttacksTool.TestNewAttacks(MyExpModel, X, y, AttacksParametersMaillard, device, FiguresPath, "NewAttackExp_C_") 

In [29]:
RNNAdversarialDataMaillardL2 = AttacksTool.MaillardL2(MyRNNModel, 0, X.float(), y.float(), 0.01, 1, 1000, device)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataMaillardL2, MyRNNModel)
print("Maillard-L2:", Infos)

GRUAdversarialDataMaillardL2 = AttacksTool.MaillardL2(MyGRUModel, 0, X.float(), y.float(), 0.01, 1, 1000, device)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataMaillardL2, MyGRUModel)
print("Maillard-L2:", Infos)

LSTMAdversarialDataMaillardL2 = AttacksTool.MaillardL2(MyLSTMModel, 0, X.float(), y.float(), 0.01, 1, 1000, device)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataMaillardL2, MyLSTMModel)
print("Maillard-L2:", Infos)

LinearAdversarialDataMaillardL2 = AttacksTool.MaillardL2(MyLinearModel, 0, X.float(), y.float(), 0.01, 1, 1000, device)
Infos = UtilsTool.GetInfos(X, y, 0, LinearAdversarialDataMaillardL2, MyLinearModel)
print("Maillard-L2:", Infos)

LogAdversarialDataMaillardL2 = AttacksTool.MaillardL2(MyLogModel, 0, X.float(), y.float(), 0.01, 1, 1000, device)
Infos = UtilsTool.GetInfos(X, y, 0, LogAdversarialDataMaillardL2, MyLogModel)
print("Maillard-L2:", Infos)

ExpAdversarialDataMaillardL2 = AttacksTool.MaillardL2(MyExpModel, 0, X.float(), y.float(), 0.01, 1, 1000, device)
Infos = UtilsTool.GetInfos(X, y, 0, ExpAdversarialDataMaillardL2, MyExpModel)
print("Maillard-L2:", Infos)

Maillard-L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 3.246049642562866, 'RMSE_adversarial': 0.02930368110537529, 'RMSE_pred': 72.78682708740234, 'L0': tensor(23.8442)}
Maillard-L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 4.1624932289123535, 'RMSE_adversarial': 0.021223703399300575, 'RMSE_pred': 72.27922821044922, 'L0': tensor(24.)}
Maillard-L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 3.2048587799072266, 'RMSE_adversarial': 0.02967594750225544, 'RMSE_pred': 72.72103881835938, 'L0': tensor(23.8182)}
Maillard-L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 0.30259037017822266, 'RMSE_adversarial': 0.014664312824606895, 'RMSE_pred': 75.63360595703125, 'L0': tensor(19.2338)}
Maillard-L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 1.169168472290039, 'RMSE_adversarial': 0.02943565882742405, 'RMSE_pred': 74.67483520507812, 'L0': tensor(23.6883)}
Maillard-L2: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 0.2

In [35]:
RNNAdversarialDataMaillardL0 = AttacksTool.MaillardL0(MyRNNModel, 0, X.float(), y.float(), 0.01, 0.01, 1000, 2, 8, 0.1, device)
Infos = UtilsTool.GetInfos(X, y, 0, RNNAdversarialDataMaillardL0, MyRNNModel)
print("Maillard-L0:", Infos)

GRUAdversarialDataMaillardL0 = AttacksTool.MaillardL0(MyGRUModel, 0, X.float(), y.float(), 0.01, 0.01, 1000, 2, 8, 0.1, device)
Infos = UtilsTool.GetInfos(X, y, 0, GRUAdversarialDataMaillardL0, MyGRUModel)
print("Maillard-L0:", Infos)

LSTMAdversarialDataMaillardL0 = AttacksTool.MaillardL0(MyLSTMModel, 0, X.float(), y.float(), 0.01, 0.01, 1000, 2, 8, 0.1, device)
Infos = UtilsTool.GetInfos(X, y, 0, LSTMAdversarialDataMaillardL0, MyLSTMModel)
print("Maillard-L0:", Infos)

LinearAdversarialDataMaillardL0 = AttacksTool.MaillardL0(MyLinearModel, 0, X.float(), y.float(), 0.006, 0.01, 1000, 2, 5, 0.1, device)
Infos = UtilsTool.GetInfos(X, y, 0, LinearAdversarialDataMaillardL0, MyLinearModel)
print("Maillard-L0:", Infos)

LogAdversarialDataMaillardL0 = AttacksTool.MaillardL0(MyLogModel, 0, X.float(), y.float(), 0.0020, 0.01, 1000, 2, 5, 0.1, device)
Infos = UtilsTool.GetInfos(X, y, 0, LogAdversarialDataMaillardL0, MyLogModel)
print("Maillard-L0:", Infos)

ExpAdversarialDataMaillardL0 = AttacksTool.MaillardL0(MyExpModel, 0, X.float(), y.float(), 0.01, 0.01, 1000, 2, 5, 0.1, device)
Infos = UtilsTool.GetInfos(X, y, 0, ExpAdversarialDataMaillardL0, MyExpModel)
print("Maillard-L0:", Infos)

Maillard-L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 6.296125411987305, 'RMSE_adversarial': 0.0682179182767868, 'RMSE_pred': 71.62680053710938, 'L0': tensor(4.6753)}
Maillard-L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 4.196709632873535, 'RMSE_adversarial': 0.048185285180807114, 'RMSE_pred': 72.90177917480469, 'L0': tensor(1.8052)}
Maillard-L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 6.8694891929626465, 'RMSE_adversarial': 0.07894039899110794, 'RMSE_pred': 70.8736343383789, 'L0': tensor(3.2987)}
Maillard-L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 0.1756773144006729, 'RMSE_adversarial': 0.02766931988298893, 'RMSE_pred': 75.89662170410156, 'L0': tensor(1.1558)}
Maillard-L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 1.9977010488510132, 'RMSE_adversarial': 0.04274965822696686, 'RMSE_pred': 75.59828186035156, 'L0': tensor(3.1688)}
Maillard-L0: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 0.519701

In [36]:
print("RNN")
PRNNAdversarialDataMaillardL2 = UtilsTool.TestProjectionCost(testloaderAttack, X, RNNAdversarialDataMaillardL2, MyRNNModel, minmax_dict, window, device)
PRNNAdversarialDataMaillardL0 = UtilsTool.TestProjectionCost(testloaderAttack, X, RNNAdversarialDataMaillardL0, MyRNNModel, minmax_dict, window, device)
print("GRU")
PGRUAdversarialDataMaillardL2 = UtilsTool.TestProjectionCost(testloaderAttack, X, GRUAdversarialDataMaillardL2, MyGRUModel, minmax_dict, window, device)
PGRUAdversarialDataMaillardL0 = UtilsTool.TestProjectionCost(testloaderAttack, X, GRUAdversarialDataMaillardL0, MyGRUModel, minmax_dict, window, device)
print("LSTM")
PLSTMAdversarialDataMaillardL2 = UtilsTool.TestProjectionCost(testloaderAttack, X, LSTMAdversarialDataMaillardL2, MyLSTMModel, minmax_dict, window, device)
PLSTMAdversarialDataMaillardL0 = UtilsTool.TestProjectionCost(testloaderAttack, X, LSTMAdversarialDataMaillardL0, MyLSTMModel, minmax_dict, window, device)
print("LOG")
PLinearAdversarialDataMaillardL2 = UtilsTool.TestProjectionCost(testloaderAttack, X, LinearAdversarialDataMaillardL2, MyLinearModel, minmax_dict, window, device)
PLinearAdversarialDataMaillardL0 = UtilsTool.TestProjectionCost(testloaderAttack, X, LinearAdversarialDataMaillardL0, MyLinearModel, minmax_dict, window, device)
print("LINEAR")
PLogAdversarialDataMaillardL2 = UtilsTool.TestProjectionCost(testloaderAttack, X, LogAdversarialDataMaillardL2, MyLogModel, minmax_dict, window, device)
PLogAdversarialDataMaillardL0 = UtilsTool.TestProjectionCost(testloaderAttack, X, LogAdversarialDataMaillardL0, MyLogModel, minmax_dict, window, device)
print("EXP")
PExpAdversarialDataMaillardL2 = UtilsTool.TestProjectionCost(testloaderAttack, X, ExpAdversarialDataMaillardL2, MyExpModel, minmax_dict, window, device)
PExpAdversarialDataMaillardL0 = UtilsTool.TestProjectionCost(testloaderAttack, X, ExpAdversarialDataMaillardL0, MyExpModel, minmax_dict, window, device)

RNN
Initial: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 3.246049642562866, 'RMSE_adversarial': 0.02930368110537529, 'RMSE_pred': 72.78682708740234, 'L0': tensor(23.8442)}
Formated: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 4.4462690353393555, 'RMSE_adversarial': 0.028818128630518913, 'RMSE_pred': 71.56489562988281, 'L0': tensor(14.7403)}

Initial: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 6.296125411987305, 'RMSE_adversarial': 0.0682179182767868, 'RMSE_pred': 71.62680053710938, 'L0': tensor(4.6753)}
Formated: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 6.295600891113281, 'RMSE_adversarial': 0.06821948289871216, 'RMSE_pred': 71.63288879394531, 'L0': tensor(4.6753)}

GRU
Initial: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 4.1624932289123535, 'RMSE_adversarial': 0.021223703399300575, 'RMSE_pred': 72.27922821044922, 'L0': tensor(24.)}
Formated: {'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 5.511565685272217, 

In [37]:
UtilsTool.CreateDFSFiles(testloaderAttack, X, PRNNAdversarialDataMaillardL2, y, MyRNNModel, minmax_dict, NormalOutputPath + "/RNN", AdversarialOutputPath + "/RNN" + "/L2", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PRNNAdversarialDataMaillardL0, y, MyRNNModel, minmax_dict, NormalOutputPath + "/RNN", AdversarialOutputPath + "/RNN" + "/L0", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, PGRUAdversarialDataMaillardL2, y, MyGRUModel, minmax_dict, NormalOutputPath + "/GRU", AdversarialOutputPath + "/GRU" + "/L2", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PGRUAdversarialDataMaillardL0, y, MyGRUModel, minmax_dict, NormalOutputPath + "/GRU", AdversarialOutputPath + "/GRU" + "/L0", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, PLSTMAdversarialDataMaillardL2, y, MyLSTMModel, minmax_dict, NormalOutputPath + "/LSTM", AdversarialOutputPath + "/LSTM" + "/L2", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PLSTMAdversarialDataMaillardL0, y, MyLSTMModel, minmax_dict, NormalOutputPath + "/LSTM", AdversarialOutputPath + "/LSTM" + "/L0", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, PLinearAdversarialDataMaillardL2, y, MyLinearModel, minmax_dict, NormalOutputPath + "/LINEAR", AdversarialOutputPath + "/LINEAR" + "/L2", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PLinearAdversarialDataMaillardL0, y, MyLinearModel, minmax_dict, NormalOutputPath + "/LINEAR", AdversarialOutputPath + "/LINEAR" + "/L0", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, PLogAdversarialDataMaillardL2, y, MyLogModel, minmax_dict, NormalOutputPath + "/LOG", AdversarialOutputPath + "/LOG" + "/L2", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PLogAdversarialDataMaillardL0, y, MyLogModel, minmax_dict, NormalOutputPath + "/LOG", AdversarialOutputPath + "/LOG" + "/L0", device)

UtilsTool.CreateDFSFiles(testloaderAttack, X, PExpAdversarialDataMaillardL2, y, MyExpModel, minmax_dict, NormalOutputPath + "/EXP", AdversarialOutputPath + "/EXP" + "/L2", device)
UtilsTool.CreateDFSFiles(testloaderAttack, X, PExpAdversarialDataMaillardL0, y, MyExpModel, minmax_dict, NormalOutputPath + "/EXP", AdversarialOutputPath + "/EXP" + "/L0", device)

In [38]:
RNNAdversarials = [PRNNAdversarialDataFgsm, PRNNAdversarialDataBim, PRNNAdversarialDataCW, PRNNAdversarialDataMaillardL2, PRNNAdversarialDataMaillardL0]
GRUAdversarials = [PGRUAdversarialDataFgsm, PGRUAdversarialDataBim, PGRUAdversarialDataCW, PGRUAdversarialDataMaillardL2, PGRUAdversarialDataMaillardL0]
LSTMAdversarials = [PLSTMAdversarialDataFgsm, PLSTMAdversarialDataBim, PLSTMAdversarialDataCW, PLSTMAdversarialDataMaillardL2, PLSTMAdversarialDataMaillardL0]

LinearAdversarials = [PLinearAdversarialDataFgsm, PLinearAdversarialDataBim, PLinearAdversarialDataCW, PLinearAdversarialDataMaillardL2, PLinearAdversarialDataMaillardL0]
LogAdversarials = [PLogAdversarialDataFgsm, PLogAdversarialDataBim, PLogAdversarialDataCW, PLogAdversarialDataMaillardL2, PLogAdversarialDataMaillardL0]
ExpAdversarials = [PExpAdversarialDataFgsm, PExpAdversarialDataBim, PExpAdversarialDataCW, PExpAdversarialDataMaillardL2, PExpAdversarialDataMaillardL0]

Adversarials = [RNNAdversarials, GRUAdversarials, LSTMAdversarials, LinearAdversarials, LogAdversarials, ExpAdversarials]

In [39]:
UtilsTool.DisplayTransferability(Adversarials, X, y, MyRNNModel, "Transferability RNN")

Transferability RNN
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 43.07719802856445, 'RMSE_adversarial': 0.05165189132094383, 'RMSE_pred': 30.74812889099121, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 23.815214157104492, 'RMSE_adversarial': 0.04548564553260803, 'RMSE_pred': 48.335350036621094, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 1.9371377229690552, 'RMSE_adversarial': 0.045177850872278214, 'RMSE_pred': 74.07440948486328, 'L0': tensor(16.0649)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 4.4462690353393555, 'RMSE_adversarial': 0.028818128630518913, 'RMSE_pred': 71.56489562988281, 'L0': tensor(14.7403)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 6.295600891113281, 'RMSE_adversarial': 0.06821948289871216, 'RMSE_pred': 71.63288879394531, 'L0': tensor(4.6753)}

{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 43.167396545410156, 'RMSE_adversarial': 0.051650188863277435, 'RM

In [40]:
UtilsTool.DisplayTransferability(Adversarials, X, y, MyGRUModel, "Transferability GRU")

Transferability GRU
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 51.27300262451172, 'RMSE_adversarial': 0.05165189132094383, 'RMSE_pred': 25.024293899536133, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 21.460615158081055, 'RMSE_adversarial': 0.04548564553260803, 'RMSE_pred': 50.142967224121094, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 10.10074234008789, 'RMSE_adversarial': 0.045177850872278214, 'RMSE_pred': 65.99554443359375, 'L0': tensor(16.0649)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 18.66899299621582, 'RMSE_adversarial': 0.028818128630518913, 'RMSE_pred': 57.11671829223633, 'L0': tensor(14.7403)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 8.905632972717285, 'RMSE_adversarial': 0.06821948289871216, 'RMSE_pred': 69.87425994873047, 'L0': tensor(4.6753)}

{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 18.954708099365234, 'RMSE_adversarial': 0.051650188863277435, 'RMS

In [41]:
UtilsTool.DisplayTransferability(Adversarials, X, y, MyLSTMModel, "Transferability LSTM")

Transferability LSTM
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 50.10506820678711, 'RMSE_adversarial': 0.05165189132094383, 'RMSE_pred': 23.947824478149414, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 27.471277236938477, 'RMSE_adversarial': 0.04548564553260803, 'RMSE_pred': 46.09392166137695, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 19.62442970275879, 'RMSE_adversarial': 0.045177850872278214, 'RMSE_pred': 52.4698371887207, 'L0': tensor(16.0649)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 27.16221809387207, 'RMSE_adversarial': 0.028818128630518913, 'RMSE_pred': 44.92425537109375, 'L0': tensor(14.7403)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 16.61229705810547, 'RMSE_adversarial': 0.06821948289871216, 'RMSE_pred': 58.87232971191406, 'L0': tensor(4.6753)}

{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 43.0158805847168, 'RMSE_adversarial': 0.051650188863277435, 'RMSE_p

In [42]:
UtilsTool.DisplayTransferability(Adversarials, X, y, MyLinearModel, "Transferability LINEAR")

Transferability LINEAR
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 57.987457275390625, 'RMSE_adversarial': 0.05165189132094383, 'RMSE_pred': 22.77495765686035, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 34.19308853149414, 'RMSE_adversarial': 0.04548564553260803, 'RMSE_pred': 35.839210510253906, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 13.502450942993164, 'RMSE_adversarial': 0.045177850872278214, 'RMSE_pred': 60.86634063720703, 'L0': tensor(16.0649)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 27.67401885986328, 'RMSE_adversarial': 0.028818128630518913, 'RMSE_pred': 42.85924530029297, 'L0': tensor(14.7403)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 10.462754249572754, 'RMSE_adversarial': 0.06821948289871216, 'RMSE_pred': 65.07477569580078, 'L0': tensor(4.6753)}

{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 17.268478393554688, 'RMSE_adversarial': 0.051650188863277435, 

In [43]:
UtilsTool.DisplayTransferability(Adversarials, X, y, MyLogModel, "Transferability LOG")

Transferability LOG
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 65.96854400634766, 'RMSE_adversarial': 0.05165189132094383, 'RMSE_pred': 24.026681900024414, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 61.45751190185547, 'RMSE_adversarial': 0.04548564553260803, 'RMSE_pred': 27.195764541625977, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 59.301902770996094, 'RMSE_adversarial': 0.045177850872278214, 'RMSE_pred': 27.545930862426758, 'L0': tensor(16.0649)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 62.266353607177734, 'RMSE_adversarial': 0.028818128630518913, 'RMSE_pred': 23.608999252319336, 'L0': tensor(14.7403)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 62.38105773925781, 'RMSE_adversarial': 0.06821948289871216, 'RMSE_pred': 25.549299240112305, 'L0': tensor(4.6753)}

{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 62.192726135253906, 'RMSE_adversarial': 0.051650188863277435, 

In [44]:
UtilsTool.DisplayTransferability(Adversarials, X, y, MyExpModel, "Transferability EXP")

Transferability EXP
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 63.297611236572266, 'RMSE_adversarial': 0.05165189132094383, 'RMSE_pred': 23.71991539001465, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 44.4518928527832, 'RMSE_adversarial': 0.04548564553260803, 'RMSE_pred': 29.038326263427734, 'L0': tensor(16.)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 44.18621826171875, 'RMSE_adversarial': 0.045177850872278214, 'RMSE_pred': 31.87576675415039, 'L0': tensor(16.0649)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 50.14548110961914, 'RMSE_adversarial': 0.028818128630518913, 'RMSE_pred': 25.182687759399414, 'L0': tensor(14.7403)}
{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 39.71366882324219, 'RMSE_adversarial': 0.06821948289871216, 'RMSE_pred': 41.59209442138672, 'L0': tensor(4.6753)}

{'RealRUL': 64.4935073852539, 'Objective': 0, 'PredRUL': 55.24876403808594, 'RMSE_adversarial': 0.051650188863277435, 'RMSE_